In [45]:
import os
import pandas as pd
import re
import unicodedata


def read_file(file_path):
    with open(file_path, "r", encoding='utf-8') as f:
        return f.read()

# Print out vocab size and contents
def print_vocab_list(data):
    chars = sorted(list(set(data)))
    vocab_size = len(chars)
    print(''.join(chars))
    print(vocab_size)

# Replace accents with normalized counterparts
def normalize_content(content):
    return unicodedata.normalize(u'NFKD', content).encode('ascii', 'ignore').decode('utf8')

# Remove punctuation, brackets, special characters
def clean_content(content):
    return re.sub(r'[^A-Za-z\s]', '', content)

file_paths = ['pg17337.txt', 'pg14269.txt', 'pg18066.txt', 'pg28085.txt', 'pg57697.txt']
contents = [read_file(file) for file in file_paths]

# Print lengths of the datasets
for i, content in enumerate(contents, 1):
    print(f"length of dataset {i} in characters: {len(content)}")

# Print vocab sizes and contents for each file
for content in contents:
    print_vocab_list(content)

normalized_contents = [normalize_content(content) for content in contents]

# Print vocab sizes and contents for each normalized file
for content in normalized_contents:
    print_vocab_list(content)

cleaned_contents = [clean_content(content) for content in normalized_contents]

# Print vocab sizes and contents for each cleaned file
for content in cleaned_contents:
    print_vocab_list(content)

lowercase_contents = [content.lower() for content in cleaned_contents]

# Concatenate all files
concatenated_content = ''.join(lowercase_contents)

# Save concatenated content to file
with open('data_final.txt', 'w', encoding='utf-8') as concatenated_file:
    concatenated_file.write(concatenated_content)

# Read the final concatenated file
with open('data_final.txt', "r", encoding='utf-8') as final_file:
    final_data = final_file.read()

print("Length of final text file is:", len(final_data))
print_vocab_list(final_data)

# Remove extra spaces
final_data = re.sub(' +', ' ', final_data).strip()


length of dataset 1 in characters: 555996
length of dataset 2 in characters: 268220
length of dataset 3 in characters: 398988
length of dataset 4 in characters: 12232
length of dataset 5 in characters: 64431

 !"'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzËàáâçèéêëíïòó﻿
92

 !"'()*,-.0123456789:;?ABCDEFGHIJKLMNOPRSTUVWZ[]_abcdefghijklmnopqrstuvwxyz àâèéêëïóôöû﻿
89

 !"'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz Ïàéêëïòó﻿
89

 !",-.05:;ABCDEFGHIKLMNOPRSTUVWZabcdefghijklmnoprstuvwxyzÈèéëïó﻿
65

 !"'()+,-./0123456789:;?ABCDEFGHIJKLMNOPRSTUVWZ\_abcdefghijklmnoprstuvwxyz|}âéêëîóôû﻿
86

 !"'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz
78

 !"'()*,-.0123456789:;?ABCDEFGHIJKLMNOPRSTUVWZ[]_abcdefghijklmnopqrstuvwxyz
76

 !"'(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz
79

 !",-.05:;ABCDEFGHIKLMNOPRSTUVWZabcdefghijklmnoprstuvwxyz
58

 !"'()+,-./0123456789:;?ABCDEFGHIJKLMNOPRST

### Now, based on this text we will train simplest GPT model . 

The method is based on Andrej Karpathy's [nanogpt video lecture](https://github.com/karpathy/ng-video-lecture/tree/master) 

The following code refers to [colab notebook](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing#scrollTo=hoelkOrFY8bN) by Andrej Karpathy

In [44]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('data_final.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.204956 M parameters
step 0: train loss 3.5044, val loss 3.4979
step 100: train loss 2.3312, val loss 2.3828
step 200: train loss 2.2565, val loss 2.3146
step 300: train loss 2.1878, val loss 2.2360
step 400: train loss 2.1104, val loss 2.1872
step 500: train loss 2.0490, val loss 2.1277
step 600: train loss 1.9954, val loss 2.0929
step 700: train loss 1.9697, val loss 2.0580
step 800: train loss 1.9439, val loss 2.0475
step 900: train loss 1.9053, val loss 2.0195
step 1000: train loss 1.8820, val loss 1.9944
step 1100: train loss 1.8656, val loss 1.9781
step 1200: train loss 1.8419, val loss 1.9822
step 1300: train loss 1.8343, val loss 1.9446
step 1400: train loss 1.8085, val loss 1.9514
step 1500: train loss 1.8042, val loss 1.9313
step 1600: train loss 1.7816, val loss 1.9210
step 1700: train loss 1.7651, val loss 1.9166
step 1800: train loss 1.7770, val loss 1.9010
step 1900: train loss 1.7551, val loss 1.9168
step 2000: train loss 1.7428, val loss 1.9017
step 2100: train loss 1.

**The following simple GPT model works on character level instead of words. The resulting generated text is incoherent, although some words and overal feel resemble the dutch language...**